In [2]:
# This should be roughly the content of the first code cell
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)

# Plotting support
from matplotlib import pyplot as plt
# from plotnine import
# Standard libraries
import pandas as pd
import sklearn as sk
import tensorflow as tf
import time
import os
from skimage import io
import cv2
from scipy import stats
from skimage.util import compare_images
from PIL import Image
from skimage.metrics import mean_squared_error, structural_similarity, hausdorff_distance, normalized_root_mse, peak_signal_noise_ratio
plt.rc('image', cmap='gray')



In [3]:
table1 = np.load('training/feature_table1.npy')
table2 = np.load('training/feature_table2.npy')
table3 = np.load('training/feature_table3.npy')
table4 = np.load('training/feature_table4.npy')
table6 = np.load('training/feature_table6.npy')
table9 = np.load('training/feature_table9.npy')
table12 = np.load('training/feature_table12.npy')
table14 = np.load('training/feature_table14.npy')
table15 = np.load('training/feature_table15.npy')
table16 = np.load('training/feature_table16.npy')

In [4]:
full_table = np.vstack((table1, table2, table3, table4, table6, table9, table12, table14, table15, table16))

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [6]:
X = np.array(full_table[:, 1:-1], dtype=float)
y = [1 if i == 'True' else 0 for i in full_table[:, -1] ]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

In [84]:
clf.predict_proba(X_train[:5])

array([[0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 2.42990290e-32],
       [1.00000000e+00, 2.97307549e-43]])

In [85]:
y_train[:5]


[0, 1, 0, 1, 0]

In [86]:
clf.predict(X_train[:5])


array([1, 1, 1, 0, 0])

In [87]:
clf.score(X_train,y_train)

0.6711330512458862

In [88]:
clf.score(X_test, y_test)

0.6863988724453841

In [89]:
from sklearn.linear_model import SGDClassifier

In [90]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [91]:
sgd_clf.score(X_train, y_train)

0.4412317818523742

In [92]:
sgd_clf.score(X_test, y_test)

0.4439746300211416

In [93]:

test1 = np.load('testing/test1_table.npy')

In [94]:
res = sgd_clf.predict(test1)
res

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [95]:
np.unique(res, return_counts=True)

(array([1]), array([12]))

In [96]:
test2 = np.load('testing/test013.npy', allow_pickle=True)

In [97]:
results2 = []
for t in test2:
    print('-------')
    # res1 = sgd_clf.predict(t[1])
    res = clf.predict(t[1])
    # res = np.hstack((res1, res2))
    print(res)
    sum = np.sum(res)
    results2.append([sum, len(res), int(t[2]), sum/len(res)])
    print('likelihood genuine: ', round(sum/len(res) * 100),'%')
    print('likelihood forged: ', round((len(res)-sum)/len(res) * 100),'%')
    print("truth ", int(t[2]))

-------
[0 0 0 0 0 1 0 0 0 0 0 0]
likelihood genuine:  8 %
likelihood forged:  92 %
truth  0
-------
[1 0 1 1 1 0 1 1 1 0 1 1]
likelihood genuine:  75 %
likelihood forged:  25 %
truth  1
-------
[1 0 0 0 0 1 1 0 1 0 0 1]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  0
-------
[0 0 0 0 1 0 1 0 0 1 0 0]
likelihood genuine:  25 %
likelihood forged:  75 %
truth  1
-------
[0 0 1 1 0 0 0 1 1 0 1 0]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  1
-------
[1 1 0 1 1 1 1 0 0 1 0 0]
likelihood genuine:  58 %
likelihood forged:  42 %
truth  0
-------
[0 0 1 1 0 0 0 1 1 0 1 1]
likelihood genuine:  50 %
likelihood forged:  50 %
truth  0
-------
[0 0 1 1 0 0 0 1 1 0 0 1]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  1
-------
[1 1 0 0 1 1 1 1 0 1 1 1]
likelihood genuine:  75 %
likelihood forged:  25 %
truth  1
-------
[1 0 0 1 1 1 1 1 1 1 1 1]
likelihood genuine:  83 %
likelihood forged:  17 %
truth  1
-------
[1 1 0 1 1 1 0 1 0 0 0 0]
likelihood genuine:  50 %
l

In [99]:
matrices_024 = np.load('testing/matrices_024.npy')
truths_024 = np.load('testing/truths_024.npy')

In [100]:
matrices_024.shape, len(truths_024)

((24, 12, 9), 24)

In [116]:
results_024 = []
count_correct  = 0

count_undefined = 0
for i, m in enumerate(matrices_024):
    res = clf.predict(m)
    sum = np.sum(res)

    if sum >= 8:
        p = 1
    elif sum <= 4:
        p = 0
    else:
        p = 2
        count_undefined += 1
    print('predicted', end=' ')
    print(p, end=' ')
    print('truth', end=' ')
    print(truths_024[i])
    if p == truths_024[i]:
        count_correct += 1

predicted 1 truth 1
predicted 0 truth 1
predicted 2 truth 0
predicted 0 truth 0
predicted 2 truth 1
predicted 0 truth 0
predicted 0 truth 0
predicted 2 truth 1
predicted 0 truth 0
predicted 2 truth 1
predicted 0 truth 0
predicted 0 truth 0
predicted 1 truth 1
predicted 2 truth 0
predicted 1 truth 1
predicted 1 truth 1
predicted 0 truth 1
predicted 0 truth 0
predicted 1 truth 0
predicted 0 truth 1
predicted 0 truth 0
predicted 1 truth 1
predicted 1 truth 1
predicted 0 truth 0


In [117]:
count_correct, count_undefined

(15, 5)

In [118]:
count_correct/ len(matrices_024)

0.625

In [119]:
count_undefined /len(matrices_024)

0.20833333333333334

In [120]:
(len(matrices_024) - count_correct - count_undefined)/len(matrices_024)

0.16666666666666666